In [1]:
import torch
from openmmtools.constants import kB
from simtk import openmm as mm
from simtk import unit
from simtk.openmm import app
from openmmtools.testsystems import AlanineDipeptideVacuum
import numpy as np
import mdtraj

import sys
# sys.path.insert(1, '/var/home/vs488/Documents/boltzmann/code/boltzmann-generators/')
sys.path.append("../")
import boltzgen.zmatrix as zmatrix
import boltzgen.internal as ics

In [1]:
# Generate a trajectory for use as training data
temperature = 298
kT = kB * temperature
testsystem = AlanineDipeptideVacuum()
vacuum_sim = app.Simulation(testsystem.topology,
                            testsystem.system,
                            mm.LangevinIntegrator(temperature * unit.kelvin , 1.0 / unit.picosecond, 1.0 * unit.femtosecond),
                            platform=mm.Platform.getPlatformByName('CPU')
                            )
vacuum_sim.context.setPositions(testsystem.positions)
vacuum_sim.minimizeEnergy()
vacuum_sim.reporters.append(mdtraj.reporters.HDF5Reporter('aldp.h5', 1000))
vacuum_sim.step(10000)
# for _ in range(10):
#    vacuum_sim.step(1)
#    traj.append(np.array(vacuum_sim.context.getState(getPositions=True).getPositions(asNumpy=True)))
# traj = np.array(traj).reshape((-1, 66))
# training_data = torch.as_tensor(traj, dtype=torch.float32)`


NameError: name 'kB' is not defined

In [2]:
aldp_traj = mdtraj.load('aldp.h5')

In [3]:
print(aldp_traj.topology.atoms_by_name)
z_mat = zmatrix.mdtraj_to_z(aldp_traj.topology, [])

<bound method Topology.atoms_by_name of <mdtraj.Topology with 1 chains, 3 residues, 22 atoms, 21 bonds at 0x1dd2ae5a288>>
residues:  [ACE1, ALA2, NME3]
molecules:  [MoleculeExtent(start_res=0, end_res=3, is_protein=True)]
residue:  ACE1
is_nterm:  True  is c_term:  False
res_atoms:  {'H1': 0, 'CH3': 1, 'H2': 2, 'H3': 3, 'C': 4, 'O': 5}
res_name:  ACE


KeyError: 'ACE'